# Exploratory Data Analysis

In [ ]:
%load_ext nb_black

In [ ]:
import scipy

import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import dates

## Data Cleaning

In [ ]:
pricing_data_df = pd.read_csv("data/test.csv")

During upfront pricing, the following factors are available to us:

1. Type of vehicle - premium, XL, go, etc
2. Customers Profile
    - Fraud Score
    - Lifetime value
    - Number of previous cancellation by driver within journey
3. Geography
    - Distance
    - Starting destination
    - Ending destination
    - Tolls
2. Traffic
    - Wait time due to incoming traffic
3. Surge
    - Time of day i.e. Rush Hoiur
    - High Demand/Low Supply
    - Bad weather

In [ ]:
pricing_data_df["calc_created"] = pd.to_datetime(pricing_data_df["calc_created"])

* Removing all UIDs and tokens as we can't feed them into model and UUIDs are calculated uniquely for each session.
* Ticket ID for resolution isn't useful as we don't have any ticket information.

In [ ]:
pricing_data_df.drop(
    ["driver_device_uid_new", "device_token", "ticket_id_new"], axis=1, inplace=True
)

Can we remove all the states if all the rides are `finished`?

In [ ]:
pricing_data_df["b_state"].value_counts()

In [ ]:
pricing_data_df["order_state"].value_counts()

In [ ]:
pricing_data_df["order_try_state"].value_counts()

In [ ]:
pricing_data_df.drop(
    ["b_state", "order_state", "order_try_state"], axis=1, inplace=True
)

All the orders are finished, hence this information is redundant.

We can remove `order_try_id_new` since we already have `order_id_new` available. Furthermore, `dest_change_number` let's us know how many times the destination was changed.

In [ ]:
pricing_data_df.drop(["order_try_id_new"], axis=1, inplace=True)

In [ ]:
pricing_data_df.drop_duplicates(inplace=True)

In [ ]:
pricing_data_df.reset_index(inplace=True, drop=True)

In [ ]:
pricing_data_df.info()

In [ ]:
upfront_pricing_data_df = pricing_data_df.loc[
    (pricing_data_df["upfront_price"].notnull()),
    :,
]

In [ ]:
upfront_pricing_data_df["prediction_price_type"].value_counts()

Since all upfront prices have prediction price type as upfront, we can drop `prediction_price_type`,

In [ ]:
upfront_pricing_data_df.drop(["prediction_price_type"], axis=1, inplace=True)

## Problem Scope

Does a deviation actually exist?

In [ ]:
upfront_pricing_data_df["upfront_price_deviation_perc"] = (
    (
        upfront_pricing_data_df["upfront_price"]
        - upfront_pricing_data_df["metered_price"]
    )
    / upfront_pricing_data_df["upfront_price"]
    * 100
)
upfront_pricing_data_df["abs_upfront_price_deviation_perc"] = abs(
    upfront_pricing_data_df["upfront_price_deviation_perc"]
)

In [ ]:
upfront_pricing_data_df["abs_upfront_price_deviation_perc"].describe(
    percentiles=[0.25, 0.5, 0.75, 0.85, 0.9, 0.95, 0.99]
)

Roughly 50% of the orders are deviating below 20% from the upfront pricing. 

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
p = sns.kdeplot(data=upfront_pricing_data_df["upfront_price_deviation_perc"], ax=ax)
x, y = p.get_lines()[0].get_data()
cdf = scipy.integrate.cumtrapz(y, x, initial=0)
nearest_05 = np.abs(cdf - 0.5).argmin()
x_median = x[nearest_05]
y_median = y[nearest_05]
plt.vlines(x_median, 0, y_median, colors="black", label="median deviation (%)")
plt.legend()
plt.xlabel("Upfront Price Deviation (%)")
plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
p = sns.kdeplot(data=upfront_pricing_data_df["abs_upfront_price_deviation_perc"], ax=ax)
x, y = p.get_lines()[0].get_data()
cdf = scipy.integrate.cumtrapz(y, x, initial=0)
nearest_05 = np.abs(cdf - 0.5).argmin()
x_median = x[nearest_05]
y_median = y[nearest_05]
plt.vlines(x_median, 0, y_median, colors="black")
plt.grid()
plt.show()

The distribution of pricing is right skewed long tailed. Our focus will on identifying what the source of the 50% of the error is.

## Predicted vs Actual Values

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
sns.kdeplot(
    data=upfront_pricing_data_df[["upfront_price", "metered_price"]], ax=ax, fill=True
)
plt.grid()
plt.show()

Upfront pricing typically seems to be a lower than the metered pricing.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
sns.kdeplot(
    data=upfront_pricing_data_df[["predicted_duration", "duration"]], ax=ax, fill=True
)
plt.grid()
plt.show()

That's because predicted duration seems to be lesser than the actual duration.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
sns.kdeplot(
    data=upfront_pricing_data_df[["predicted_distance", "distance"]], ax=ax, fill=True
)
plt.grid()
plt.show()

The same goes for distance. the predicted distance typically lays on the lower end.

There's a common theme where the algorithm tends to under-estimate the pricing, distance and duration.

## Problem Impact

How many customers does this deviation impact?

In [ ]:
pricing_data_df.shape

Assuming our population consists of 4270 customers.

In [ ]:
upfront_pricing_data_df.shape[0] / pricing_data_df.shape[0]

In [ ]:
upfront_pricing_data_df.shape[0]

Around 70% of the customers have suffered from some form of a deviation between upfront and metered pricing on the app.

In [ ]:
upfront_pricing_data_df["upfront_price_deviation_perc"].describe(
    percentiles=[0.25, 0.35, 0.5, 0.55, 0.75, 0.85, 0.9, 0.95, 0.99]
)

In [ ]:
upfront_pricing_data_df[
    upfront_pricing_data_df["upfront_price_deviation_perc"] < 0
].shape[0] / upfront_pricing_data_df.shape[0]

Around 60% of the customers see a price higher than the one that is shown upfront.

In [ ]:
upfront_pricing_data_df[
    upfront_pricing_data_df["upfront_price_deviation_perc"] < -20
].shape[0] / upfront_pricing_data_df.shape[0]

Around 35% of the customers get charged more at the end of the journey.

We're going to assume that anyone who created an `overpaid_ride_ticket` and didn't pay more for a ride, did it by accident. 

In [ ]:
upfront_pricing_data_df[
    (upfront_pricing_data_df["upfront_price_deviation_perc"] < -20)
]["overpaid_ride_ticket"].value_counts(normalize=True)

4% of customers who were shown a higher price (i.e. with a deviation of 20%), complained about an overpaid ticket.

## Identifying the source of the deviation

What factor is causing the price to deviate more than usual? When is the deviation occuring and by how much?

In [ ]:
upfront_pricing_data_df["abs_upfront_price_deviation_perc"] = abs(
    (
        upfront_pricing_data_df["upfront_price"]
        - upfront_pricing_data_df["metered_price"]
    )
    / upfront_pricing_data_df["upfront_price"]
    * 100
)

### GPS Confidence

Is the GPS confidence poor which is why we're seeing such a high deviation?

In [ ]:
gps_price_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["gps_confidence"])[
        "abs_upfront_price_deviation_perc"
    ]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "median": "Median Abs Deviation (%)",
        "std": "Standard Deviation",
    }
)
gps_price_abs_deviation_df["Coefficient of Variation"] = (
    gps_price_abs_deviation_df["Standard Deviation"]
    / gps_price_abs_deviation_df["Mean Abs Deviation (%)"]
)

In [ ]:
gps_price_abs_deviation_df

* There's a significant increase in both the mean and median deviation of pricing when the GPS confidence is poor.
* Could this be a result of the device someone is utilizing?

In [ ]:
upfront_pricing_data_df["device_manufacturer"] = (
    upfront_pricing_data_df["device_name"].str.split(" |,|\-|\_").str[0].str.lower()
)
upfront_pricing_data_df["device_manufacturer"] = upfront_pricing_data_df[
    "device_manufacturer"
].apply(lambda x: "iphone" if "iphone" in x else x)
upfront_pricing_data_df["device_manufacturer"] = upfront_pricing_data_df[
    "device_manufacturer"
].apply(lambda x: "tecno" if "tecno" in x else x)

Which device manufacturer is returning the highest gps confidence error?

In [ ]:
gps_low_conf_device_perc_df = pd.merge(
    upfront_pricing_data_df.groupby(["device_manufacturer"])["gps_confidence"]
    .count()
    .reset_index()
    .rename(columns={"gps_confidence": "Num Devices"}),
    upfront_pricing_data_df[upfront_pricing_data_df["gps_confidence"] == 0]
    .groupby(["device_manufacturer"])["gps_confidence"]
    .count()
    .reset_index()
    .rename(columns={"gps_confidence": "Num Devices with 0 GPS conf"}),
    on="device_manufacturer",
    how="left",
).fillna(0)

gps_low_conf_device_perc_df["% 0 GPS conf devices"] = (
    gps_low_conf_device_perc_df["Num Devices with 0 GPS conf"]
    * 100.0
    / gps_low_conf_device_perc_df["Num Devices"]
)

In [ ]:
gps_low_conf_device_perc_df.sort_values(["% 0 GPS conf devices"], ascending=False).head(
    15
)

In [ ]:
gps_low_conf_device_perc_df[
    gps_low_conf_device_perc_df["Num Devices"] >= 10
].sort_values(["% 0 GPS conf devices"], ascending=False).head(15)

Brands like `infinix`, `itel`, `tecno` seem to be the culprit here, as they're resulting in a high percentage of device with poor GPS confidence.

### Device Manufacturer

In [ ]:
device_price_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["device_manufacturer"])[
        "abs_upfront_price_deviation_perc"
    ]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "median": "Median Abs Deviation (%)",
    }
)

In [ ]:
device_price_abs_deviation_df[device_price_abs_deviation_df["count"] > 10].sort_values(
    "Median Abs Deviation (%)", ascending=False
)

In line with the points with from the poor GPS confidence, brands like `infinix`, `itel`, `tecno` also result in a high median deviation and might require additional calibration.

### Date

Is there any particular day that the prices are deviation more often?

In [ ]:
upfront_pricing_data_df["ride_date"] = upfront_pricing_data_df["calc_created"].dt.date

In [ ]:
date_price_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["ride_date"])["abs_upfront_price_deviation_perc"]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "ride_date": "Ride Date",
        "median": "Median Abs Deviation (%)",
    }
)
date_price_abs_deviation_df["Coefficient of Variation"] = (
    date_price_abs_deviation_df["std"]
    / date_price_abs_deviation_df["Mean Abs Deviation (%)"]
)
melted_date_price_abs_deviation_df = pd.melt(
    date_price_abs_deviation_df,
    value_vars=["Median Abs Deviation (%)", "Mean Abs Deviation (%)"],
    id_vars="Ride Date",
)

In [ ]:
melted_date_price_abs_deviation_df

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
g = sns.lineplot(
    data=melted_date_price_abs_deviation_df,
    x="Ride Date",
    y="value",
    hue="variable",
    marker="o",
    ax=ax,
)
plt.xticks(rotation=90)
ax.set(xticks=melted_date_price_abs_deviation_df["Ride Date"].values)
plt.ylabel("Deviation (%)")
g.axhline(
    upfront_pricing_data_df["abs_upfront_price_deviation_perc"].mean(),
    linestyle="--",
    color="orange",
    label="Global Mean Abs Deviation (%)",
)
g.axhline(
    upfront_pricing_data_df["abs_upfront_price_deviation_perc"].median(),
    linestyle="--",
    color="blue",
    label="Global Median Abs Deviation (%)",
)
plt.legend()
plt.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6), dpi=120)
sns.lineplot(
    data=date_price_abs_deviation_df,
    x="Ride Date",
    y="Coefficient of Variation",
    marker="o",
    ax=ax,
)
plt.xticks(rotation=90)
ax.set(xticks=date_price_abs_deviation_df["Ride Date"].values)
plt.grid()

In [ ]:
upfront_pricing_data_df.loc[
    upfront_pricing_data_df["ride_date"]
    == dt.datetime(year=2020, month=2, day=4).date(),
    ["abs_upfront_price_deviation_perc"],
].describe()

* Although the mean deviation is higher on the 16th, the median is still roughly similar to the global median. 
* There is an outlier on the 16th that's resulting in the higher mean price deviation. Furthermore, the coefficient of variation is the highest on that date.
* Date isn't a factor that's directly impacting the upfront price deviation.

In [ ]:
upfront_pricing_data_df.loc[
    (
        upfront_pricing_data_df["ride_date"]
        == dt.datetime(year=2020, month=2, day=4).date()
    )
    & (upfront_pricing_data_df["abs_upfront_price_deviation_perc"] >= 300),
].T

There's a significant difference in the duration and distance here, which is why we're seeing such a mean deviation.

## Destination Changes

Changing destinations midway through the ride could require the algorithm to re-calculate the journey.

In [ ]:
dest_price_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["dest_change_number"])[
        "abs_upfront_price_deviation_perc"
    ]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "median": "Median Abs Deviation (%)",
        "std": "Standard Deviation",
    }
)
dest_price_abs_deviation_df["Coefficient of Variation"] = (
    dest_price_abs_deviation_df["Standard Deviation"]
    / dest_price_abs_deviation_df["Mean Abs Deviation (%)"]
)

In [ ]:
dest_price_abs_deviation_df

* There's a significant increase in the median deviation above 1 ride changes. 
* However it's difficult to say with confidence that this trend will continue as the number of ride changes increase.
* We can probably show the customer an alert stating that pricing would change drastically with each ride change.

## EU Indicator

In [ ]:
eu_price_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["eu_indicator"])[
        "abs_upfront_price_deviation_perc"
    ]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "median": "Median Abs Deviation (%)",
        "std": "Standard Deviation",
    }
)
eu_price_abs_deviation_df["Coefficient of Variation"] = (
    eu_price_abs_deviation_df["Standard Deviation"]
    / eu_price_abs_deviation_df["Mean Abs Deviation (%)"]
)

In [ ]:
eu_price_abs_deviation_df

* There is a much higher delta in non-EU countries
* This could be because the road infrastructure is far better in the EU. Frequent road closures, deviation in distances cause prices to increase.
* It could also be due to regulations put in place that do not allow price deviations in the EU.

In [ ]:
upfront_pricing_data_df["abs_distance_deviation_perc"] = abs(
    (
        upfront_pricing_data_df["predicted_distance"]
        - upfront_pricing_data_df["distance"]
    )
    / upfront_pricing_data_df["predicted_distance"]
    * 100
)

In [ ]:
eu_distance_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["eu_indicator"])["abs_distance_deviation_perc"]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "median": "Median Abs Deviation (%)",
        "std": "Standard Deviation",
    }
)
eu_distance_abs_deviation_df["Coefficient of Variation"] = (
    eu_distance_abs_deviation_df["Standard Deviation"]
    / eu_distance_abs_deviation_df["Mean Abs Deviation (%)"]
)

In [ ]:
eu_distance_abs_deviation_df

Since deviation in distances and deviation in prices is correlated, it's evident that distances deviate more likely in non-EU countries as opposed to EU ones.

## App version

Is there any particular app version that's buggy?

Although pricing is calculated on the backend, is there something inherently wrong with a way a certain app version extracts location or duration information?

In [ ]:
ride_appv_price_abs_deviation_df = (
    upfront_pricing_data_df.groupby(["rider_app_version"])[
        "abs_upfront_price_deviation_perc"
    ]
    .agg(["mean", "count", "median", "std"])
    .reset_index()
).rename(
    columns={
        "mean": "Mean Abs Deviation (%)",
        "median": "Median Abs Deviation (%)",
        "std": "Standard Deviation",
    }
)
ride_appv_price_abs_deviation_df["Coefficient of Variation"] = (
    ride_appv_price_abs_deviation_df["Standard Deviation"]
    / ride_appv_price_abs_deviation_df["Mean Abs Deviation (%)"]
)

In [ ]:
ride_appv_price_abs_deviation_df[
    ride_appv_price_abs_deviation_df["count"] > 10
].sort_values(by=["Median Abs Deviation (%)"], ascending=False).head(25)

* It seems like certain app versions like `CA.5.47` are producing higher Median abs deviation than others and might need to be looked into.
* Since app versions depend on adoption, we can check to see the percentage of rides on an app version with at least 20% deviation.

In [ ]:
ride_appv_price_deviation_perc_df = (
    pd.merge(
        upfront_pricing_data_df.groupby(["rider_app_version"])["order_id_new"]
        .count()
        .reset_index(),
        upfront_pricing_data_df[
            upfront_pricing_data_df["abs_upfront_price_deviation_perc"] > 20
        ]
        .groupby(["rider_app_version"])["order_id_new"]
        .count()
        .reset_index(),
        on=["rider_app_version"],
        how="left",
    )
    .fillna(0)
    .rename(
        columns={
            "order_id_new_x": "num_rides",
            "order_id_new_y": "num_rides_with_20_perc_deviation",
        }
    )
)

In [ ]:
ride_appv_price_deviation_perc_df["perc_rides_with_20_perc_deviation"] = (
    ride_appv_price_deviation_perc_df["num_rides_with_20_perc_deviation"]
    * 100
    / ride_appv_price_deviation_perc_df["num_rides"]
)

In [ ]:
ride_appv_price_deviation_perc_df[
    ride_appv_price_deviation_perc_df["num_rides"] > 10
].sort_values(by=["perc_rides_with_20_perc_deviation"], ascending=False)

Similar to the insight above, app versions like `CA.5.47` are producing a high number of rides with errors.

## 0 Distances

In [ ]:
upfront_pricing_data_df.loc[
    upfront_pricing_data_df["distance"] == 0, "abs_upfront_price_deviation_perc"
].describe()

In [ ]:
upfront_pricing_data_df.loc[
    upfront_pricing_data_df["duration"] == 0, ["distance", "duration"]
]

In [ ]:
upfront_pricing_data_df.loc[
    upfront_pricing_data_df["duration"] == 0, ["distance", "duration"]
].shape